<a href="https://colab.research.google.com/github/junghyeonsu/Algorithm_Application/blob/master/ViterbiSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Viterbi Search

a) 왜 이 주제가 중요한지?

b) 우리 일상생활에 어떻게 적용되는지?

c) 활용가능한 주요 알고리즘의 이름, 패키지 명

d) 해당 알고리즘 혹은 주제의 핵심 아이디어

# Viterbi Search는 왜 중요할까?

Viterbi Search는 자연어 처리에 대표적인 알고리즘으로 우리의 일상생활에서 나타나는 자연어를 컴퓨터가 처리할 수 있도록 하는 기술이다.

기계에게 인간의 언어를 이해시킨다는 점에서 인공지능에 있어서 가장 중요한 연구 분야이면서도, 아직 정복되지 못한 산이다.

# 일상생활에서는 어떻게 적용될까?

자연어 처리는 

1. 음성 인식

2. 내용 요약

3. 번역

4. 사용자의 감성 분석

5. 텍스트 분류 작업(스팸 메일, 뉴스 기사 카테고리 분류)

6. 질의 응답 시스템 

7. 챗봇

과 같은 곳에서 사용된다.
우리의 언어와 관련이 있으면 대부분 사용된다.

# Viterbi Search

Viterbi Search의 대표적인 알고리즘으로써

Hidden Markov model을 제공해주는 Sklearn 패키지가 있다.

# Viterbi Search의 핵심 아이디어

![image](https://user-images.githubusercontent.com/54893898/83946899-a822f480-a84e-11ea-9219-392095be993e.png)


Viterbi Search의 핵심 아이디어로는

우선 자연어 처리중의 품사 판별기에 대해서 예를 들겠다.

Janet will back the bill 이라는 문장이 들어오면 각각의 단어가 어떤 품사를 가지고있는지 판별을 하는것이다.

품사는 총 7분류로 나뉘어지고 총 5개의 단어가 각각 7개의 품사에 대해서 점수가 매겨진다.

그러면 그 중 가장 높은 점수인것을 찾는다.

그리고 그 다음 단어로 가서 전까지 계산되었던 것과 지금 판별하고있는 단어의 점수를 조합해 가장 높은 길을 또 찾는다. 

위의 사진 같은 경우에는 Janet은 NNP에 점수가 가장높았고, will 같은 경우에는 Janet이 NNP였던 점수와 will이 MD일지, VB일지, NN일지 전부 계산해서 가장 높은 것인 MD를 고른 것이다.

이런식으로 반복해서 지금 선택하는 것의 점수와 지금까지 왔던 길의 점수를 조합해 가장 높은 길을 계속해서 찾아나간다.

그러면 총 5개의 단어의 품사를 모두 고르게 되는데 이때 지나왔던 길을 다시 돌아가면 이 문장에 대한 품사가 전부 판별이 되는것이다.

In [0]:
# 아래는 데이터 셋이다.
#                 NNP      MD      VB      JJ       NN      RB     DT
initial_probs = [0.2767, 0.0006, 0.0031, 0.0453, 0.0449, 0.0510, 0.2026]

transition_probs = [ #  NNP      MD      VB      JJ       NN      RB     DT
                      [0.3777, 0.0110, 0.0009, 0.0084, 0.0584, 0.0090, 0.0025], # NNP
                      [0.0008, 0.0002, 0.7968, 0.0005, 0.0008, 0.1698, 0.0041], # MD
                      [0.0322, 0.0005, 0.0050, 0.0837, 0.0615, 0.0514, 0.2231], # VB
                      [0.0366, 0.0004, 0.0001, 0.0733, 0.4509, 0.0036, 0.0036], # JJ
                      [0.0096, 0.0176, 0.0014, 0.0086, 0.1216, 0.0177, 0.0068], # NN
                      [0.0068, 0.0102, 0.1011, 0.1012, 0.0120, 0.0728, 0.0479], # RB
                      [0.1147, 0.0021, 0.0002, 0.2157, 0.4744, 0.0102, 0.0017]  # DT
                    ]

observation_probs = [# JANET       WILL      BACK      THE        BILL
                      [0.000032, 0.000000, 0.000000, 0.000048, 0.000000], # NNP
                      [0.000000, 0.308431, 0.000000, 0.000000, 0.000000], # MD
                      [0.000000, 0.000028, 0.000672, 0.000000, 0.000028], # VB
                      [0.000000, 0.000000, 0.000340, 0.000000, 0.000000], # JJ
                      [0.000000, 0.000200, 0.000223, 0.000000, 0.002337], # NN
                      [0.000000, 0.000000, 0.010446, 0.000000, 0.000000], # RB
                      [0.000000, 0.000000, 0.000000, 0.506099, 0.000000], # DT
                    ]

states = ['NNP', 'MD', 'VB', 'JJ', 'NN', 'RB', 'DT']    


def viterbi_edu_not_efficient(transition, observation, init_probs):
  _, T = observation.shape
  _, N = transition.shape

  V = np.zeros((N,T)) # viterbi matrix
  B = np.zeros((N,T), dtype=np.int) # backtrace points

  # initialization step
  # print("initialization step -----------------------------")
  for s in range(N):
    V[s][0] = init_probs[s] * observation[s][0]
    B[s][0] = 0
    # print("V[{}][{}]={:.6f}*{:.6f}={:.6f}".format(s,0,
    #                                               init_probs[s],
    #                                               observation[s][0],
    #                                               V[s][0]
    #                                               )
    # )

    # print("recursion step ------------------------------")
    # Recursion step
    # 돌아가면서 가장 높은 확률이 어떤 것인지 고르는 작업이다.
    for t in range(1,T):
      # print("Time : ", t)
      for s in range(N):
        _values = []
        for prev_s in range(N):
          _values.append( V[prev_s][t-1] * transition[prev_s][s] )

        s_prime = np.argmax(_values)
        max_viterbi_value = _values[s_prime]

        prob = max_viterbi_value * observation[s][t]
        V[s][t] = prob

        # print("V[{}][{}]=V[{}][{}] = {:.6f}*{:.6f}={}".format(s,t, states[s],t,
        #                                                       max_viterbi_value,
        #                                                       observation[s][t],
        #                                                       V[s][t],
        #                                                       V[s][t]
        #                                                       )
        # )

        B[s][t] = int(s_prime)

  # 최고인 Path를 출력하기위한 작업이다.
  # print("termination step -------------------------")
  best_last_state = np.argmax(V[:,T-1])
  best_path_prob = V[best_last_state, T-1]

  # 걸어왔던 길을 다시 돌아가면서 어떤것이 best였는지 확인한다.
  best_path = [best_last_state]
  for t in reversed( range(1, T) ):
    prev_best_state = B[best_last_state][t]
    best_path.append( prev_best_state)
    best_last_state = prev_best_state

  best_path = reversed(best_path)
  best_tag_seq = [ states[i] for i in best_path]

  return best_tag_seq, best_path_prob


# 아래 메소드는 위의 곱셈을 했던 것들을 로그 덧셈으로 바꾼것이다.
def viterbi_edu_log_not_efficient(transition, observation, init_probs):
  _, T = observation.shape
  _, N = transition.shape

  V = np.zeros((N,T)) # viterbi matrix
  B = np.zeros((N,T), dtype=np.int) # backtrace points

  # initialization step
  # print("initialization step -----------------------------")
  for s in range(N):
    V[s][0] = np.log(init_probs[s]) +  np.log(observation[s][0])
    B[s][0] = 0
    # print("V[{}][{}]={:.6f}*{:.6f}={:.6f}".format(s,0,
    #                                               init_probs[s],
    #                                               observation[s][0],
    #                                               V[s][0]
    #                                               )
    # )

    # print("recursion step ------------------------------")
    # Recursion step
    for t in range(1,T):
      # print("Time : ", t)
      for s in range(N):
        _values = []
        for prev_s in range(N):
          _values.append( V[prev_s][t-1] + np.log(transition[prev_s][s]) )

        s_prime = np.argmax(_values)
        max_viterbi_value = _values[s_prime]

        prob = max_viterbi_value + np.log(observation[s][t])
        V[s][t] = prob

        # print("V[{}][{}]=V[{}][{}] = {:.6f}*{:.6f}={}".format(s,t, states[s],t,
        #                                                       max_viterbi_value,
        #                                                       observation[s][t],
        #                                                       V[s][t],
        #                                                       V[s][t]
        #                                                       )
        # )

        B[s][t] = int(s_prime)

  # print("termination step -------------------------")
  best_last_state = np.argmax(V[:,T-1])
  best_path_log_prob = V[best_last_state, T-1]
  best_path_prob = np.exp( best_path_log_prob )

  best_path = [best_last_state]
  for t in reversed( range(1, T) ):
    prev_best_state = B[best_last_state][t]
    best_path.append( prev_best_state)
    best_last_state = prev_best_state

  best_path = reversed(best_path)
  best_tag_seq = [ states[i] for i in best_path]

  return best_tag_seq, best_path_prob


import numpy as np

trans = np.array(transition_probs)
obs   = np.array(observation_probs)
inits = np.array(initial_probs)

best_tag_seq, best_path_prob = viterbi_edu_not_efficient(trans, obs, inits)
print(best_tag_seq, best_path_prob)

best_tag_seq, best_path_prob = viterbi_edu_log_not_efficient(trans, obs, inits)
print(best_tag_seq, best_path_prob)












['NNP', 'MD', 'VB', 'DT', 'NN'] 2.013570710221386e-15
['NNP', 'MD', 'VB', 'DT', 'NN'] 2.0135707102213835e-15


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:119: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:98: RuntimeWarning: divide by zero encountered in log
